In [20]:
import requests
import json
from gtts import gTTS
import gradio as gr
import tempfile

OPENROUTER_API_KEY = "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"

def generate_lyrics(title):
    prompt = (
        f"एक बॉलीवुड गीत लिखिए जिसका शीर्षक '{title}' हो। "
        "यह गीत रोमांटिक और भावनात्मक हो, जिसमें मुखड़ा और अंतरा शामिल हों।"
    )
    response = requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": "deepseek/deepseek-r1:free",
            "messages": [{"role": "user", "content": prompt}]
        })
    )

    if response.status_code != 200:
        # <—— now returns only the error string
        return f"Error {response.status_code}: {response.text}"

    lyrics = response.json()["choices"][0]["message"]["content"]
    return lyrics


def lyrics_to_audio(lyrics):
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    gTTS(text=lyrics, lang="hi").save(tmp_file.name)
    return tmp_file.name

def generate_song_from_title(title):
    lyrics = generate_lyrics(title)
    if lyrics.startswith("Error:"):
        return lyrics, None
    audio_path = lyrics_to_audio(lyrics)
    return lyrics, audio_path

with gr.Blocks() as demo:
    gr.Markdown("## 🎵 Task Title to Bollywood Song + Audio Generator")

    title_input = gr.Textbox(label="Enter Task Title (Song Title)", placeholder="e.g. प्यार की बारिश", lines=1)
    generate_btn = gr.Button("Generate Song")

    lyrics_output = gr.Textbox(label="Generated Song Lyrics", lines=12)
    audio_output = gr.Audio(label="Song Audio", type="filepath")

    generate_btn.click(
        fn=generate_song_from_title,
        inputs=title_input,
        outputs=[lyrics_output, audio_output]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
